In [59]:
# Load the GEO Data 

In [60]:
import GEOparse
import pandas as pd
import sklearn.cluster

In [61]:
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [62]:
gse = GEOparse.get_GEO(filepath="./GSE10245_family.soft.gz")

28-Nov-2022 15:05:04 INFO GEOparse - Parsing ./GSE10245_family.soft.gz: 
28-Nov-2022 15:05:04 DEBUG GEOparse - DATABASE: GeoMiame
28-Nov-2022 15:05:04 DEBUG GEOparse - SERIES: GSE10245
28-Nov-2022 15:05:04 DEBUG GEOparse - PLATFORM: GPL570
/opt/anaconda3/lib/python3.9/site-packages/GEOparse/GEOparse.py:401: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  return read_csv(StringIO(data), index_col=None, sep="\t")
28-Nov-2022 15:05:05 DEBUG GEOparse - SAMPLE: GSM258551
28-Nov-2022 15:05:05 DEBUG GEOparse - SAMPLE: GSM258552
28-Nov-2022 15:05:05 DEBUG GEOparse - SAMPLE: GSM258553
28-Nov-2022 15:05:06 DEBUG GEOparse - SAMPLE: GSM258554
28-Nov-2022 15:05:06 DEBUG GEOparse - SAMPLE: GSM258555
28-Nov-2022 15:05:06 DEBUG GEOparse - SAMPLE: GSM258556
28-Nov-2022 15:05:06 DEBUG GEOparse - SAMPLE: GSM258557
28-Nov-2022 15:05:06 DEBUG GEOparse - SAMPLE: GSM258558
28-Nov-2022 15:05:06 DEBUG GEOparse - SAMPLE: GSM258559
28-Nov-2022 15:05:06 DEBUG 

In [66]:
matrix_data = gse.pivot_samples("VALUE").T

In [67]:
charac_list = []
for gsm in gse.gsms.values():
    charac_list.append(gsm.metadata["characteristics_ch1"][0].split(":")[1][1:])

In [68]:
# Below are part 4,5,6

In [69]:
# Accuracy of training data with model trained on training data 
def compute_accuracy(Train_data,Train_subtype,random_state_num): 
    #develope the kmeans model
    kmeans = KMeans(n_clusters = 2, random_state = random_state_num).fit(Train_data)
    
    # Predict label(in the form of array)
    predict_label = kmeans.labels_
    # convert predict label array into list 
    predict_label = predict_label.tolist()
    
    # Create a dataframe for predict label and subtype
    df = pd.DataFrame([predict_label,Train_subtype]).T
    df.columns = ["Cluster", "Subtype"]
    
    # Retrive the subtype with cluster == 1
    df_1 = df.loc[df['Cluster'] == 1]
    # Find the most frequent subtype 
    sub_1 = df_1['Subtype'].mode().iloc[0]
    
    # Retrive the subtype with cluster == 0
    df_0 = df.loc[df['Cluster'] == 0]
    # Find the most frequent subtype 
    sub_0 = df_0['Subtype'].mode().iloc[0]
    
    # Define the subtype and corresponding labels
    cancer_subtype = {sub_0: 0,sub_1: 1}
    
    # convert train_cancer_type into train_label
    train_label = [cancer_subtype[item] for item in Train_subtype]
    # Compute accuracy_score 
    return accuracy_score(train_label, predict_label)

In [70]:
# Accuracy of the model trained on all data 
compute_accuracy(matrix_data,charac_list,0)

0.8793103448275862

In [71]:
# Split all data into 50% train and 50% test
X_train, X_test, y_train, y_test = train_test_split(matrix_data,charac_list,test_size=0.5)

In [72]:
# Accuracy of the model trained on 50% of training data 
compute_accuracy(X_train,y_train,0)

0.9310344827586207

In [73]:
# Accuracy of testing data with model trained on training data 
def compute_test_accuracy(Data,subtype,random_state_num):
    
    # Split all data into 50% train and 50% test
    X_train, X_test, y_train, y_test = train_test_split(matrix_data,charac_list,test_size=0.5)
    
    # Develop kmeans model trained based on training data 
    kmeans = KMeans(n_clusters = 2, random_state = random_state_num).fit(X_train)
    
    # Predict label for testing data
    predict_label = kmeans.fit_predict(X_test)

    # Predict label for testing data
    predict_label = kmeans.fit_predict(X_test)
    # convert predict label array into list 
    predict_label = predict_label.tolist()

    # Create a dataframe for predict label and subtype(test label)
    df = pd.DataFrame([predict_label,y_test]).T
    df.columns = ["Cluster", "Subtype"]

    # Retrive the subtype with cluster == 1
    df_1 = df.loc[df['Cluster'] == 1]
    # Find the most frequent subtype 
    sub_1 = df_1['Subtype'].mode().iloc[0]

    # Retrive the subtype with cluster == 0
    df_0 = df.loc[df['Cluster'] == 0]
    # Find the most frequent subtype 
    sub_0 = df_0['Subtype'].mode().iloc[0]

    # Define the subtype and corresponding labels
    cancer_subtype = {sub_0: 0,sub_1: 1}

    # convert train_cancer_type into train_label
    train_label = [cancer_subtype[item] for item in y_test]
    
    return accuracy_score(train_label, predict_label)

In [74]:
compute_test_accuracy(matrix_data,charac_list,0)

0.9655172413793104